In [ ]:
import pandas as pd

data = pd.read_csv('computed_10.0.csv')

In [ ]:
data = data.rename(columns={"Unnamed: 0": "index","owener":"owner"})
data

,index,additions,deletions,number_of_patches_so_far,initial_files_changed,max_files_addition,max_files_deletion,max_files_delta,max_files_size,commit_sbj,commit_msg,entropy,files_touched,has_topic,topic,topic_prevalency,mergeable,has_assignee,owner_is_submitter,project,project_age,project_prev_commits,created,updated,subimi,eval_time,status,average_NUR,owner,dev_prev_exp,dev_merged_CR,change_id,id,decision_made
0,0,3,1,2,2,3,1,33,14430,BACKPORT: ASoC: Intel: Clear stored Baytrail D...,BACKPORT: ASoC: Intel: Clear stored Baytrail D...,0.000000,3,False,NaN,0.0,NaN,False,False,chromiumos/third_party/kernel-next,776.828067,8846,1.402969e+09,1.402971e+09,1.402969e+09,2160.0,MERGED,29.000000,1108879,23,3,Id3da8374728d4333325e8d20561a6972df24e2de,chromiumos%2Fthird_party%2Fkernel-next~release...,True
1,1,9,5,1,1,5,3,69,18887,gen_msvs_proj: fix in tree configure under cygwin,gen_msvs_proj: fix in tree configure under cyg...,0.985228,1,False,NaN,0.0,NaN,False,True,webm/libvpx,791.469433,5466,1.402969e+09,1.403073e+09,1.402969e+09,104250.0,MERGED,18.500000,1002219,1008,15,I76208e6cdc66dc5a0a7ffa8aa1edbefe31e4b130,webm%2Flibvpx~master~I76208e6cdc66dc5a0a7ffa8a...,True
2,2,5,2,5,5,5,2,60,746,minimal: Disable power management services,minimal: Disable power management services\n\n...,0.000000,4,False,NaN,0.0,NaN,False,False,chromiumos/overlays/chromiumos-overlay,794.848877,5330,1.402970e+09,1.403078e+09,1.402970e+09,108753.0,MERGED,3.000000,1000071,207,23,If5c89e7fa8fd5a7a23d95f5662cdbd74c48f1574,chromiumos%2Foverlays%2Fchromiumos-overlay~mas...,True
3,3,0,0,1,1,0,0,0,4190776,Update arm gcc toolchain 4.8's gdb version to 7.6,Update arm gcc toolchain 4.8's gdb version to ...,0.000000,1,False,NaN,0.0,NaN,False,False,android_tools,593.894572,65,1.402970e+09,1.402976e+09,1.402970e+09,6713.0,MERGED,3.000000,1115910,0,0,I6dbffb1ab6d6d5c4c508746e57f25207720d1db3,android_tools~master~I6dbffb1ab6d6d5c4c508746e...,True
4,4,9,6,2,2,3,2,35,7506,Fix issues with feedback logs,Fix issues with feedback logs\n\nThere were is...,1.000000,1,False,NaN,0.0,NaN,False,False,chromiumos/platform/inputcontrol,794.216227,98,1.402970e+09,1.403078e+09,1.402970e+09,108374.0,MERGED,4.666667,1001564,434,14,I30136ee78c7d62a0704658fce694ae41396f3e2b,chromiumos%2Fplatform%2Finputcontrol~master~I3...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,133,78,4,4,118,65,1635,18868,CHROMIUM: thermal: tegra-soctherm: use functio...,CHROMIUM: thermal: tegra-soctherm: use functio...,0.413027,3,False,NaN,0.0,NaN,False,False,chromiumos/third_party/kernel,876.037280,7172,1.409993e+09,1.411216e+09,1.409993e+09,1223259.0,MERGED,13.333333,1000376,181,36,Ie92520c34186c27112f937306b15f35724437da2,chromiumos%2Fthird_party%2Fkernel~chromeos-3.1...,True
9996,9996,2,1,4,4,2,1,35,23046,CHROMIUM: ARM64: DTS: tegra132: add interrup l...,CHROMIUM: ARM64: DTS: tegra132: add interrup l...,0.000000,1,False,NaN,0.0,NaN,False,False,chromiumos/third_party/kernel,876.037280,7173,1.409993e+09,1.411554e+09,1.409993e+09,1561739.0,MERGED,33.000000,1000376,182,37,I902c5003f682243f201ec3e3ee66d2226ff3f115,chromiumos%2Fthird_party%2Fkernel~chromeos-3.1...,True
9997,9997,638,5,4,4,638,5,18480,73459,CHROMIUM: thermal: tegra-soctherm: support con...,CHROMIUM: thermal: tegra-soctherm: support con...,0.000000,1,False,NaN,0.0,NaN,False,False,chromiumos/third_party/kernel,876.037280,7174,1.409993e+09,1.411554e+09,1.409993e+09,1561732.0,MERGED,23.000000,1000376,183,38,I33ebcd8e447ac604921e4558299818c43f09ec73,chromiumos%2Fthird_party%2Fkernel~chromeos-3.1...,True
9998,9998,2,2,3,3,2,2,0,1693,beaglebone: update spi pinmux settings,beaglebone: update spi pinmux settings\n\n* Re...,0.000000,3,False,NaN,0.0,NaN,False,False,chromiumos/overlays/board-overlays,872.809931,1136,1.409996e+09,1.410372e+09,1.409996e+09,376552.0,MERGED,0.500000,1000104,10,3,I054db14a7795cd222d9cef61a042575a460af543,chromiumos%2Foverlays%2Fboard-

In [ ]:
# train_data = data.drop
only_determined = data[data['decision_made']==True].drop(['subimi','change_id','id'],axis=1)


In [ ]:
import re
def denoiseTokenize(sentence):
    sentence = re.sub(r"[\*\"“”\n\\…\+\/\=\(\)‘•:\[\]\|’\!;><]", " ", str(sentence))
    sentence = re.sub(r"[ ]+", " ", sentence)
    sentence = re.sub(r"\!+", "!", sentence)
    sentence = re.sub(r"\,+", ",", sentence)
    sentence = re.sub(r"\?+", "?", sentence)
    sentence = re.sub(r"\&+", " ", sentence)
    sentence = re.sub(r"\'s", " ", sentence)
    sentence = sentence.lower()
    return sentence
only_determined['commit_sbj'] = only_determined['commit_sbj'].apply(denoiseTokenize)
only_determined['commit_msg'] = only_determined['commit_msg'].apply(denoiseTokenize)


In [ ]:
projects_list = list(only_determined.project.unique())
project_to_index = {}
index_to_project = {}

def make_category():
  for project,i in enumerate(projects_list):
    project_to_index[i] = project
    index_to_project[project] = i
  return project_to_index, index_to_project
project_to_index, index_to_project = make_category()


def convert_project_to_code(row):
  # print(row)
  row['project_code'] = project_to_index[row['project']]
  return row
only_determined = only_determined.apply(convert_project_to_code,axis=1)

In [ ]:
projects_list = list(only_determined.project.unique())
project_to_index = {}
index_to_project = {}

def make_category():
  for project,i in enumerate(projects_list):
    project_to_index[i] = project
    index_to_project[project] = i
  return project_to_index, index_to_project
project_to_index, index_to_project = make_category()


def convert_project_to_code(row):
  # print(row)
  row['project_code'] = project_to_index[row['project']]
  return row
only_determined = only_determined.apply(convert_project_to_code,axis=1).drop(['project'], axis=1)

In [ ]:
only_determined['topic'] = only_determined['topic'].fillna(-1)

topics_list = list(only_determined.topic.unique())
topic_to_index = {}
index_to_topic = {}

def make_topic_categories():
  for topic,i in enumerate(topics_list):
    topic_to_index[i] = topic
    index_to_topic[topic] = i
  return topic_to_index, index_to_topic
topic_to_index, index_to_topic = make_topic_categories()


def convert_topic_to_code(row):
  row['topic_code'] = topic_to_index[row['topic']]
  return row
only_determined = only_determined.apply(convert_topic_to_code,axis=1).drop(['topic'], axis=1)

only_determined

,index,additions,deletions,number_of_patches_so_far,initial_files_changed,max_files_addition,max_files_deletion,max_files_delta,max_files_size,commit_sbj,commit_msg,entropy,files_touched,has_topic,topic_prevalency,mergeable,has_assignee,owner_is_submitter,project_age,project_prev_commits,created,updated,eval_time,status,average_NUR,owner,dev_prev_exp,dev_merged_CR,decision_made,project_code,topic_code
0,0,3,1,2,2,3,1,33,14430,backport asoc intel clear stored baytrail dsp ...,backport asoc intel clear stored baytrail dsp ...,0.000000,3,False,0.0,NaN,False,False,776.828067,8846,1.402969e+09,1.402971e+09,2160.0,MERGED,29.000000,1108879,23,3,True,0,0
1,1,9,5,1,1,5,3,69,18887,gen_msvs_proj fix in tree configure under cygwin,gen_msvs_proj fix in tree configure under cygw...,0.985228,1,False,0.0,NaN,False,True,791.469433,5466,1.402969e+09,1.403073e+09,104250.0,MERGED,18.500000,1002219,1008,15,True,1,0
2,2,5,2,5,5,5,2,60,746,minimal disable power management services,minimal disable power management services bug ...,0.000000,4,False,0.0,NaN,False,False,794.848877,5330,1.402970e+09,1.403078e+09,108753.0,MERGED,3.000000,1000071,207,23,True,2,0
3,3,0,0,1,1,0,0,0,4190776,update arm gcc toolchain 4.8 gdb version to 7.6,update arm gcc toolchain 4.8 gdb version to 7...,0.000000,1,False,0.0,NaN,False,False,593.894572,65,1.402970e+09,1.402976e+09,6713.0,MERGED,3.000000,1115910,0,0,True,3,0
4,4,9,6,2,2,3,2,35,7506,fix issues with feedback logs,fix issues with feedback logs there were issue...,1.000000,1,False,0.0,NaN,False,False,794.216227,98,1.402970e+09,1.403078e+09,108374.0,MERGED,4.666667,1001564,434,14,True,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,133,78,4,4,118,65,1635,18868,chromium thermal tegra-soctherm use functions ...,chromium thermal tegra-soctherm use functions ...,0.413027,3,False,0.0,NaN,False,False,876.037280,7172,1.409993e+09,1.411216e+09,1223259.0,MERGED,13.333333,1000376,181,36,True,6,0
9996,9996,2,1,4,4,2,1,35,23046,chromium arm64 dts tegra132 add interrup line ...,chromium arm64 dts tegra132 add interrup line ...,0.000000,1,False,0.0,NaN,False,False,876.037280,7173,1.409993e+09,1.411554e+09,1561739.0,MERGED,33.000000,1000376,182,37,True,6,0
9997,9997,638,5,4,4,638,5,18480,73459,chromium thermal tegra-soctherm support config...,chromium thermal tegra-soctherm support config...,0.000000,1,False,0.0,NaN,False,False,876.037280,7174,1.409993e+09,1.411554e+09,1561732.0,MERGED,23.000000,1000376,183,38,True,6,0
9998,9998,2,2,3,3,2,2,0,1693,beaglebone update spi pinmux settings,beaglebone update spi pinmux settings reverse ...,0.000000,3,False,0.0,NaN,False,False,872.809931,1136,1.409996e+09,1.410372e+09,376552.0,MERGED,0.500000,1000104,10,3,True,24,0


In [ ]:
status_list = list(only_determined.status.unique())
status_to_index = {}
index_to_status = {}

def make_status_categories():
  for status,i in enumerate(status_list):
    status_to_index[i] = status
    index_to_status[status] = i
  return status_to_index, index_to_status
status_to_index, index_to_status = make_status_categories()


def convert_status_to_code(row):
  row['status_code'] = status_to_index[row['status']]
  return row
only_determined = only_determined.apply(convert_status_to_code,axis=1).drop(['status'], axis=1)

In [ ]:
owner_list = list(only_determined.owner.unique())
owner_to_index = {}
index_to_owner = {}

def make_owner_categories():
  for owner,i in enumerate(owner_list):
    owner_to_index[i] = owner
    index_to_owner[owner] = i
  return owner_to_index, index_to_owner
owner_to_index, index_to_owner = make_owner_categories()


def convert_owner_to_code(row):
  row['owner_code'] = owner_to_index[row['owner']]
  return row
only_determined = only_determined.apply(convert_owner_to_code,axis=1).drop(['owner'], axis=1)

In [ ]:
only_determined['mergeable'] = only_determined['mergeable'].fillna(-1)

In [ ]:
only_determined

,index,additions,deletions,number_of_patches_so_far,initial_files_changed,max_files_addition,max_files_deletion,max_files_delta,max_files_size,commit_sbj,commit_msg,entropy,files_touched,has_topic,topic_prevalency,mergeable,has_assignee,owner_is_submitter,project_age,project_prev_commits,created,updated,eval_time,average_NUR,dev_prev_exp,dev_merged_CR,decision_made,project_code,topic_code,status_code,owner_code
0,0,3,1,2,2,3,1,33,14430,backport asoc intel clear stored baytrail dsp ...,backport asoc intel clear stored baytrail dsp ...,0.000000,3,False,0.0,-1.0,False,False,776.828067,8846,1.402969e+09,1.402971e+09,2160.0,29.000000,23,3,True,0,0,0,0
1,1,9,5,1,1,5,3,69,18887,gen_msvs_proj fix in tree configure under cygwin,gen_msvs_proj fix in tree configure under cygw...,0.985228,1,False,0.0,-1.0,False,True,791.469433,5466,1.402969e+09,1.403073e+09,104250.0,18.500000,1008,15,True,1,0,0,1
2,2,5,2,5,5,5,2,60,746,minimal disable power management services,minimal disable power management services bug ...,0.000000,4,False,0.0,-1.0,False,False,794.848877,5330,1.402970e+09,1.403078e+09,108753.0,3.000000,207,23,True,2,0,0,2
3,3,0,0,1,1,0,0,0,4190776,update arm gcc toolchain 4.8 gdb version to 7.6,update arm gcc toolchain 4.8 gdb version to 7...,0.000000,1,False,0.0,-1.0,False,False,593.894572,65,1.402970e+09,1.402976e+09,6713.0,3.000000,0,0,True,3,0,0,3
4,4,9,6,2,2,3,2,35,7506,fix issues with feedback logs,fix issues with feedback logs there were issue...,1.000000,1,False,0.0,-1.0,False,False,794.216227,98,1.402970e+09,1.403078e+09,108374.0,4.666667,434,14,True,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,133,78,4,4,118,65,1635,18868,chromium thermal tegra-soctherm use functions ...,chromium thermal tegra-soctherm use functions ...,0.413027,3,False,0.0,-1.0,False,False,876.037280,7172,1.409993e+09,1.411216e+09,1223259.0,13.333333,181,36,True,6,0,0,240
9996,9996,2,1,4,4,2,1,35,23046,chromium arm64 dts tegra132 add interrup line ...,chromium arm64 dts tegra132 add interrup line ...,0.000000,1,False,0.0,-1.0,False,False,876.037280,7173,1.409993e+09,1.411554e+09,1561739.0,33.000000,182,37,True,6,0,0,240
9997,9997,638,5,4,4,638,5,18480,73459,chromium thermal tegra-soctherm support config...,chromium thermal tegra-soctherm support config...,0.000000,1,False,0.0,-1.0,False,False,876.037280,7174,1.409993e+09,1.411554e+09,1561732.0,23.000000,183,38,True,6,0,0,240
9998,9998,2,2,3,3,2,2,0,1693,beaglebone update spi pinmux settings,beaglebone update spi pinmux settings reverse ...,0.000000,3,False,0.0,-1.0,False,False,872.809931,1136,1.409996e+09,1.410372e+09,376552.0,0.500000,10,3,True,24,0,0,423


In [ ]:
def separate_data(data):
  text_columns = ['commit_sbj','commit_msg']
  clmns = data.columns
  for textual_clmn in text_columns:
    clmns = clmns[clmns!=textual_clmn]

  target_info = ['eval_time']
  for target_clmn in target_info:
    clmns = clmns[clmns!=target_clmn]

  future_clmns = ['number_of_patches_so_far','status','decision_made']
  for future_clmn in future_clmns:
    clmns = clmns[clmns!=future_clmn]

  numerical_inputs = data[clmns].astype('float')
  textual_inputs = data[text_columns]
  target = data[target_info]
  return textual_inputs, numerical_inputs, target

In [ ]:
!pip install transformers

In [ ]:
import torch.nn as nn
import torch.optim as optim
# from adabelief_pytorch import AdaBelief


import torch.nn.functional as F

from transformers import BertTokenizer, BertModel
import torch


class Predictor(nn.Module):
    def __init__(self,p=0.3):
        super(Predictor, self).__init__()

        self.bert=BertModel.from_pretrained('bert-base-uncased')
        self.ff1=nn.Linear(2*768+24,256)
        self.ff2=nn.Linear(256,64)
        self.ff3=nn.Linear(64,1)
        self.dropout = nn.Dropout(p=p)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


    def change_bert_trainable(self,val):
      # print(self.bert.parameters())
      for param in self.bert.parameters():
        param.requires_grad = val

        
    def forward(self,text,other):
      bert_input1=self.tokenizer(text[0], return_tensors="pt",add_special_tokens=True, truncation=True)
      bert_out1=self.bert(bert_input1['input_ids'].cuda())
      bert_input2=self.tokenizer(text[1], return_tensors="pt",add_special_tokens=True, truncation=True)
      bert_out2=self.bert(bert_input2['input_ids'].cuda())

      other_tens=other
      ff1_out = F.relu(self.ff1(torch.cat((bert_out1.last_hidden_state[0][0],
                                         bert_out2.last_hidden_state[0][0],other_tens))))
      ff2_out = self.dropout(F.relu(self.ff2(ff1_out)))
      # ff2_out = F.relu(self.ff2(ff1_out))
      ff3_out = self.ff3(ff2_out)
      return ff3_out
      

      







In [ ]:
# optimizer=optim.Adam(model.parameters(),lr=0.005)


# for epoch in range(20):
#   model.zero_grad()
#   text='I made some changes.'
#   other=[1,2,0.1,10]


#   gold=torch.tensor([15.5],dtype=torch.float)
#   out=model(text,other)

#   loss=loss_function(gold,out)


#   loss.backward()

#   optimizer.step()




# model.eval()

# out=model(text,other)




In [ ]:
def convert_target_to_hour(row):
  row['eval_time'] = row['eval_time']/3600
  return row
only_determined = only_determined.apply(convert_target_to_hour,axis=1).drop(['created','updated'], axis=1)

KeyError: ignored

In [ ]:
bptt = 1

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    return data

In [ ]:
import time
from tqdm import tqdm 

model=Predictor().cuda()
#model.change_bert_trainable(False)


criterion = nn.MSELoss()#nn.CrossEntropyLoss()
lr = 0.005 # learning rate
# optimizer = AdaBelief(model.parameters())
optimizer=optim.Adam(model.parameters(),lr=lr)
save_round=100
all_losses = []
def train(data,epoch):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    textual_inputs, numerical_inputs, targets = separate_data(data)
    textual_inputs = (textual_inputs.values.tolist())
    print(len(textual_inputs))
    print(textual_inputs)
    numerical_inputs = numerical_inputs.values.tolist()
    targets = targets.values.tolist()
    for batch, i in tqdm(enumerate(range(0, only_determined.shape[0] - 1, 1)),total=only_determined.shape[0], disable=True):
        optimizer.zero_grad()
        numerical_inpt=torch.tensor(numerical_inputs[i],dtype=torch.float).cuda()
        output = model(textual_inputs[i],numerical_inpt)
        target= torch.tensor(targets[i],dtype=torch.float).cuda()
        loss = criterion(output, target)
        # print(output, targets[i], loss)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        log_interval = 200
        cur_loss = total_loss
        elapsed = time.time() - start_time
        if i % save_round == 0:
          print('\n| epoch ',epoch, '| {:5d}/{:5d} \t| {:5.2f}\t | loss {:5.2f}'
                .format(i+1, len(data) // bptt, elapsed * 1000 ,total_loss / save_round))
          all_losses.append(total_loss / save_round)
          total_loss = 0
    total_loss = 0
    start_time = time.time()
    return all_losses

In [ ]:
# print(only_determined.iloc[0])
epochs_losses = []
epochs =15
for i in range(epochs):
 epochs_losses.append(train(only_determined,i))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  0 |     1/ 9896 	| 346.11	 | loss 96.87

| epoch  0 |   101/ 9896 	| 9807.05	 | loss 68129269.03

| epoch  0 |   201/ 9896 	| 18636.99	 | loss 17818482.82

| epoch  0 |   301/ 9896 	| 27712.98	 | loss 49872795.74

| epoch  0 |   401/ 9896 	| 36956.93	 | loss 41270358.38

| epoch  0 |   501/ 9896 	| 45993.12	 | loss 28687803.03

| epoch  0 |   601/ 9896 	| 54857.06	 | loss 39548603.97

| epoch  0 |   701/ 9896 	| 63836.43	 | loss 7814410.41

| epoch  0 |   801/ 9896 	| 73027.61	 | loss 63435260.44

| epoch  0 |   901/ 9896 	| 82212.45	 | loss 4418889.18

| epoch  0 |  1001/ 9896 	| 91987.12	 | loss 38751055.20

| epoch  0 |  1101/ 9896 	| 101326.79	 | loss 90828797.89

| epoch  0 |  1201/ 9896 	| 110665.68	 | loss 459091722.01

| epoch  0 |  1301/ 9896 	| 120124.69	 | loss 69851593731.83

| epoch  0 |  1401/ 9896 	| 128951.69	 | loss 596792692.35

| epoch  0 |  1501/ 9896 	| 138272.25	 | loss 2903310.49

| epoch  0 |  1601/ 9896 	| 147418.95	 | loss 39572266016.27

| epoch  0 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  1 |   101/ 9896 	| 9672.65	 | loss 100843909.35

| epoch  1 |   201/ 9896 	| 19287.37	 | loss 6280251.96

| epoch  1 |   301/ 9896 	| 28243.82	 | loss 5168472.90

| epoch  1 |   401/ 9896 	| 37819.72	 | loss 36551704.69

| epoch  1 |   501/ 9896 	| 47169.43	 | loss 29133528.98

| epoch  1 |   601/ 9896 	| 56874.21	 | loss 62217323.31

| epoch  1 |   701/ 9896 	| 66125.37	 | loss 8312128.24

| epoch  1 |   801/ 9896 	| 75668.43	 | loss 62145119.81

| epoch  1 |   901/ 9896 	| 84778.80	 | loss 33414417.86

| epoch  1 |  1001/ 9896 	| 94289.04	 | loss 39475833.67

| epoch  1 |  1101/ 9896 	| 103612.88	 | loss 96999038.92

| epoch  1 |  1201/ 9896 	| 112695.05	 | loss 57482435.99

| epoch  1 |  1301/ 9896 	| 122289.24	 | loss 24828606860.03

| epoch  1 |  1401/ 9896 	| 131763.36	 | loss 64069717.39

| epoch  1 |  1501/ 9896 	| 141027.39	 | loss 1546454.26

| epoch  1 |  1601/ 9896 	| 150492.08	 | loss 28456938223.29

| epoch  1 |  1701/ 9896 	| 159704.31	 | loss 33606829.08

| ep

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...


| epoch  12 |   101/ 9896 	| 9380.94	 | loss 13338456.58

| epoch  12 |   201/ 9896 	| 18387.32	 | loss 29584520.54

| epoch  12 |   301/ 9896 	| 27169.23	 | loss 146729.58

| epoch  12 |   401/ 9896 	| 35983.43	 | loss 25280374.73

| epoch  12 |   501/ 9896 	| 45409.63	 | loss 28863596.47

| epoch  12 |   601/ 9896 	| 54133.65	 | loss 38867605.09

| epoch  12 |   701/ 9896 	| 63315.84	 | loss 7824031.94

| epoch  12 |   801/ 9896 	| 72687.00	 | loss 63783885.24

| epoch  12 |   901/ 9896 	| 81969.19	 | loss 3642280.43

| epoch  12 |  1001/ 9896 	| 91155.85	 | loss 38407837.75

| epoch  12 |  1101/ 9896 	| 100641.24	 | loss 90752228.63

| epoch  12 |  1201/ 9896 	| 109744.39	 | loss 58637985.92

| epoch  12 |  1301/ 9896 	| 119679.57	 | loss 48139900.11

| epoch  12 |  1401/ 9896 	| 129064.44	 | loss 51578351.83

| epoch  12 |  1501/ 9896 	| 138487.73	 | loss 687599.42

| epoch  12 |  1601/ 9896 	| 148065.11	 | loss 76977.02

| epoch  12 |  1701/ 9896 	| 157805.14	 | loss 6651486.98



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  13 |   101/ 9896 	| 9481.86	 | loss 13330755.86

| epoch  13 |   201/ 9896 	| 19172.84	 | loss 29565511.84

| epoch  13 |   301/ 9896 	| 28748.99	 | loss 145224.10

| epoch  13 |   401/ 9896 	| 38095.10	 | loss 25263152.08

| epoch  13 |   501/ 9896 	| 47435.11	 | loss 28841893.38

| epoch  13 |   601/ 9896 	| 56815.71	 | loss 38838552.01

| epoch  13 |   701/ 9896 	| 65911.72	 | loss 7817592.72

| epoch  13 |   801/ 9896 	| 74901.66	 | loss 63745223.73

| epoch  13 |   901/ 9896 	| 84084.24	 | loss 3633452.19

| epoch  13 |  1001/ 9896 	| 93084.59	 | loss 38376703.98

| epoch  13 |  1101/ 9896 	| 102247.94	 | loss 90695920.23

| epoch  13 |  1201/ 9896 	| 111475.90	 | loss 58601802.15

| epoch  13 |  1301/ 9896 	| 120695.63	 | loss 48101694.77

| epoch  13 |  1401/ 9896 	| 129732.88	 | loss 51547676.57

| epoch  13 |  1501/ 9896 	| 139046.75	 | loss 685746.48

| epoch  13 |  1601/ 9896 	| 148037.91	 | loss 75501.04

| epoch  13 |  1701/ 9896 	| 156996.05	 | loss 6637882.12



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  14 |     1/ 9896 	| 334.42	 | loss 65.05

| epoch  14 |   101/ 9896 	| 9626.87	 | loss 13323325.14

| epoch  14 |   201/ 9896 	| 18385.78	 | loss 29546868.80

| epoch  14 |   301/ 9896 	| 27277.84	 | loss 143931.54

| epoch  14 |   401/ 9896 	| 36210.65	 | loss 25246281.50

| epoch  14 |   501/ 9896 	| 45159.13	 | loss 28820578.01

| epoch  14 |   601/ 9896 	| 54447.47	 | loss 38809963.66

| epoch  14 |   701/ 9896 	| 63712.57	 | loss 7811411.52

| epoch  14 |   801/ 9896 	| 72930.49	 | loss 63707113.00

| epoch  14 |   901/ 9896 	| 81931.05	 | loss 3624902.43

| epoch  14 |  1001/ 9896 	| 90982.78	 | loss 38346053.07

| epoch  14 |  1101/ 9896 	| 99896.81	 | loss 90640322.20

| epoch  14 |  1201/ 9896 	| 108755.61	 | loss 58566138.89

| epoch  14 |  1301/ 9896 	| 117919.50	 | loss 48064026.03

| epoch  14 |  1401/ 9896 	| 127023.66	 | loss 51517475.38

| epoch  14 |  1501/ 9896 	| 136447.75	 | loss 684109.37

| epoch  14 |  1601/ 9896 	| 145970.04	 | loss 74237.27

| epoch  

In [ ]:
model.change_bert_trainable(True)
epochs =3
lr=0.001
optimizer=optim.Adam(model.parameters(),lr=lr)
for i in range(epochs):
 all_losses = train(only_determined,i)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  0 |   101/ 9896 	| 9629.58	 | loss 13316196.05

| epoch  0 |   201/ 9896 	| 18923.68	 | loss 29528776.35

| epoch  0 |   301/ 9896 	| 28309.95	 | loss 142863.88

| epoch  0 |   401/ 9896 	| 37476.58	 | loss 25230096.86

| epoch  0 |   501/ 9896 	| 46807.58	 | loss 28800250.26

| epoch  0 |   601/ 9896 	| 56295.57	 | loss 38782935.15

| epoch  0 |   701/ 9896 	| 65402.93	 | loss 7805758.67

| epoch  0 |   801/ 9896 	| 74486.61	 | loss 63671658.18

| epoch  0 |   901/ 9896 	| 83394.91	 | loss 3617184.11

| epoch  0 |  1001/ 9896 	| 92975.19	 | loss 38318095.39

| epoch  0 |  1101/ 9896 	| 102319.03	 | loss 90590396.53

| epoch  0 |  1201/ 9896 	| 111565.32	 | loss 58534849.88

| epoch  0 |  1301/ 9896 	| 120874.98	 | loss 48031440.59

| epoch  0 |  1401/ 9896 	| 130128.03	 | loss 51491719.96

| epoch  0 |  1501/ 9896 	| 139498.10	 | loss 682873.00

| epoch  0 |  1601/ 9896 	| 148890.97	 | loss 73319.03

| epoch  0 |  1701/ 9896 	| 158302.27	 | loss 6613492.23

| epoch  0 |  180

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  1 |   101/ 9896 	| 9157.76	 | loss 13314809.06

| epoch  1 |   201/ 9896 	| 18443.82	 | loss 29525214.36

| epoch  1 |   301/ 9896 	| 27464.02	 | loss 142672.89

| epoch  1 |   401/ 9896 	| 36378.43	 | loss 25226868.18

| epoch  1 |   501/ 9896 	| 45452.29	 | loss 28796163.99

| epoch  1 |   601/ 9896 	| 54667.27	 | loss 38777427.13

| epoch  1 |   701/ 9896 	| 64119.03	 | loss 7804612.70

| epoch  1 |   801/ 9896 	| 73353.44	 | loss 63664289.55

| epoch  1 |   901/ 9896 	| 83002.57	 | loss 3615573.60

| epoch  1 |  1001/ 9896 	| 92285.47	 | loss 38312169.10

| epoch  1 |  1101/ 9896 	| 101808.60	 | loss 90579581.02

| epoch  1 |  1201/ 9896 	| 111022.40	 | loss 58527917.68

| epoch  1 |  1301/ 9896 	| 120363.50	 | loss 48024106.88

| epoch  1 |  1401/ 9896 	| 129286.83	 | loss 51485844.21

| epoch  1 |  1501/ 9896 	| 138484.86	 | loss 682607.33

| epoch  1 |  1601/ 9896 	| 148069.14	 | loss 73126.84

| epoch  1 |  1701/ 9896 	| 157359.95	 | loss 6610935.20

| epoch  1 |  180

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




| epoch  2 |     1/ 9896 	| 327.06	 | loss 89.06

| epoch  2 |   101/ 9896 	| 9472.96	 | loss 13313416.59

| epoch  2 |   201/ 9896 	| 18741.97	 | loss 29521613.24

| epoch  2 |   301/ 9896 	| 28288.38	 | loss 142487.44

| epoch  2 |   401/ 9896 	| 37840.63	 | loss 25223615.83

| epoch  2 |   501/ 9896 	| 47345.59	 | loss 28792035.05

| epoch  2 |   601/ 9896 	| 57217.91	 | loss 38771864.86

| epoch  2 |   701/ 9896 	| 67524.69	 | loss 7803463.29

| epoch  2 |   801/ 9896 	| 77245.11	 | loss 63656848.13

| epoch  2 |   901/ 9896 	| 86642.03	 | loss 3613956.86

| epoch  2 |  1001/ 9896 	| 95980.75	 | loss 38306196.09

| epoch  2 |  1101/ 9896 	| 105161.28	 | loss 90568691.53

| epoch  2 |  1201/ 9896 	| 114244.48	 | loss 58520957.81

| epoch  2 |  1301/ 9896 	| 123457.59	 | loss 48016749.67

| epoch  2 |  1401/ 9896 	| 132686.64	 | loss 51479955.96

| epoch  2 |  1501/ 9896 	| 142175.82	 | loss 682348.89

| epoch  2 |  1601/ 9896 	| 151419.73	 | loss 72941.95

| epoch  2 |  1701/ 9896 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save(model.state_dict(), './initial_result.model')


In [ ]:
import pandas as pd

data = pd.read_csv('computed_11.0.csv')

In [ ]:
data = data.rename(columns={"Unnamed: 0": "index","owener":"owner"})
data

,index,additions,deletions,number_of_patches_so_far,initial_files_changed,max_files_addition,max_files_deletion,max_files_delta,max_files_size,commit_sbj,commit_msg,entropy,files_touched,has_topic,topic,topic_prevalency,mergeable,has_assignee,owner_is_submitter,project,project_age,project_prev_commits,created,updated,subimi,eval_time,status,average_NUR,owner,dev_prev_exp,dev_merged_CR,change_id,id,decision_made
0,0,2,5,2,2,2,5,0,2843,tegra132: fix compilation error,tegra132: fix compilation error\n\nTwo commits...,0.000000,1,False,NaN,0.0,NaN,False,False,chromiumos/third_party/coreboot,515.242245,3574,1.409997e+09,1.410232e+09,1.409997e+09,235082.0,MERGED,3.000000,1002124,640,0,I72a268fe1e1e06a6eaacf955320786180de3b070,chromiumos%2Fthird_party%2Fcoreboot~chromeos-2...,True
1,1,21,0,1,1,17,0,722,1504,sys-apps/findutils: Install a single version o...,sys-apps/findutils: Install a single version o...,0.702467,1,False,NaN,0.0,NaN,False,False,chromiumos/overlays/portage-stable,875.324375,748,1.409999e+09,1.410237e+09,1.409999e+09,238163.0,ABANDONED,0.000000,1002279,308,9,Ie4b2434bf31d5dadbbe0593e0492c2e8dc900fa2,chromiumos%2Foverlays%2Fportage-stable~master~...,True
2,2,40,36,1,1,22,35,668,50796,make VP8LSetBitPos() set br->eos_ flag,make VP8LSetBitPos() set br->eos_ flag\n\nRead...,0.588432,1,False,NaN,0.0,NaN,False,True,webm/libwebp,875.904201,1371,1.410000e+09,1.410000e+09,1.410000e+09,155.0,MERGED,45.666667,1000898,431,16,I2d2a275f38834ba005bc767d45c5de72d032103e,webm%2Flibwebp~master~I2d2a275f38834ba005bc767...,True
3,3,4,0,2,2,4,0,126,62508,UPSTREAM: ieee80211: add definition for 802.11...,UPSTREAM: ieee80211: add definition for 802.11...,0.000000,1,False,NaN,0.0,NaN,False,False,chromiumos/third_party/kernel,876.148218,7175,1.410002e+09,1.410311e+09,1.410002e+09,308512.0,MERGED,3.000000,1002363,1166,69,I30f941e09087839fb862f8936f54375778967c9a,chromiumos%2Fthird_party%2Fkernel~chromeos-3.1...,True
4,4,89,0,3,3,77,0,2066,2066,ddccontrol: import newer version,ddccontrol: import newer version\n\nThis is th...,0.395174,3,False,NaN,0.0,NaN,False,False,chromiumos/overlays/portage-stable,875.362002,749,1.410003e+09,1.410285e+09,1.410003e+09,282442.0,MERGED,0.666667,1000768,2125,13,I9ba9f4e29573a61ef41a264e7d0a7de20fe85b5d,chromiumos%2Foverlays%2Fportage-stable~master~...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,315,1,2,2,266,1,7484,8537,charge_manager: Add unit tests,charge_manager: Add unit tests\n\nAdd unit tes...,0.354284,1,False,NaN,0.0,NaN,False,False,chromiumos/platform/ec,946.725104,4778,1.416098e+09,1.416113e+09,1.416098e+09,15191.0,MERGED,41.166667,1103909,296,43,I31962588ca7360e2ffde6b83459505872e2128b9,chromiumos%2Fplatform%2Fec~firmware-samus-6300...,True
9996,9996,0,20,2,2,0,13,0,61396,update_engine: Remove unused methods from Upda...,update_engine: Remove unused methods from Upda...,0.934068,2,False,NaN,0.0,NaN,False,False,chromiumos/platform2,260.838414,1344,1.416098e+09,1.416299e+09,1.416098e+09,201517.0,MERGED,21.500000,1002279,377,27,I1e869c259855df50c6fe04f1b2ab2c5fdfc79b80,chromiumos%2Fplatform2~master~I1e869c259855df5...,True
9997,9997,82,9,4,4,73,9,2083,10690,[autotest] Allow non-test provision actions li...,[autotest] Allow non-test provision actions li...,0.751032,2,False,NaN,0.0,NaN,False,False,chromiumos/third_party/autotest,946.434387,8648,1.416112e+09,1.534285e+09,1.416112e+09,118172970.0,MERGED,7.000000,1001834,138,9,I3b449238b823fbede9d8583de9adcb5254bac23c,chromiumos%2Fthird_party%2Fautotest~master~I3b...,True
9998,9998,16,0,5,5,16,0,365,16553,CHROMIUM: ARM: dts: veyron: Add ts3a227e heads...,CHROMIUM: ARM: dts: veyron: Add ts3a227e heads...,0.000000,5,False,NaN,0.0,NaN,False,False,chromiumos/third_party/kernel,946.975521,10088,1.416122e+09,1.416448e+09,1.416122e+09,326707.0,MERGED,31.000000,1000994,1502,110,Ia503b3b8ad644dee1e0a225c9749508c065ad807,chromiumos%2Fthird_party%2Fkernel~chromeo

In [ ]:
# train_data = data.drop
only_determined = data[data['decision_made']==True].drop(['subimi','change_id','id'],axis=1)


In [ ]:
projects_list = list(only_determined.project.unique())
project_to_index = {}
index_to_project = {}

def make_category():
  for project,i in enumerate(projects_list):
    project_to_index[i] = project
    index_to_project[project] = i
  return project_to_index, index_to_project
project_to_index, index_to_project = make_category()


def convert_project_to_code(row):
  # print(row)
  row['project_code'] = project_to_index[row['project']]
  return row
only_determined = only_determined.apply(convert_project_to_code,axis=1)

In [ ]:
projects_list = list(only_determined.project.unique())
project_to_index = {}
index_to_project = {}

def make_category():
  for project,i in enumerate(projects_list):
    project_to_index[i] = project
    index_to_project[project] = i
  return project_to_index, index_to_project
project_to_index, index_to_project = make_category()


def convert_project_to_code(row):
  # print(row)
  row['project_code'] = project_to_index[row['project']]
  return row
only_determined = only_determined.apply(convert_project_to_code,axis=1).drop(['project'], axis=1)

In [ ]:
import re
def denoiseTokenize(sentence):
    sentence = re.sub(r"[\*\"“”\n\\…\+\/\=\(\)‘•:\[\]\|’\!;><]", " ", str(sentence))
    sentence = re.sub(r"[ ]+", " ", sentence)
    sentence = re.sub(r"\!+", "!", sentence)
    sentence = re.sub(r"\,+", ",", sentence)
    sentence = re.sub(r"\?+", "?", sentence)
    sentence = re.sub(r"\&+", " ", sentence)
    sentence = re.sub(r"\'s", " ", sentence)
    sentence = sentence.lower()
    return sentence
only_determined['commit_sbj'] = only_determined['commit_sbj'].apply(denoiseTokenize)
only_determined['commit_msg'] = only_determined['commit_msg'].apply(denoiseTokenize)


In [ ]:
only_determined['topic'] = only_determined['topic'].fillna(-1)

topics_list = list(only_determined.topic.unique())
topic_to_index = {}
index_to_topic = {}

def make_topic_categories():
  for topic,i in enumerate(topics_list):
    topic_to_index[i] = topic
    index_to_topic[topic] = i
  return topic_to_index, index_to_topic
topic_to_index, index_to_topic = make_topic_categories()


def convert_topic_to_code(row):
  row['topic_code'] = topic_to_index[row['topic']]
  return row
only_determined = only_determined.apply(convert_topic_to_code,axis=1).drop(['topic'], axis=1)

only_determined

,index,additions,deletions,number_of_patches_so_far,initial_files_changed,max_files_addition,max_files_deletion,max_files_delta,max_files_size,commit_sbj,commit_msg,entropy,files_touched,has_topic,topic_prevalency,mergeable,has_assignee,owner_is_submitter,project_age,project_prev_commits,created,updated,eval_time,status,average_NUR,owner,dev_prev_exp,dev_merged_CR,decision_made,project_code,topic_code
0,0,2,5,2,2,2,5,0,2843,tegra132 fix compilation error,tegra132 fix compilation error two commits wer...,0.000000,1,False,0.0,NaN,False,False,515.242245,3574,1.409997e+09,1.410232e+09,235082.0,MERGED,3.000000,1002124,640,0,True,0,0
1,1,21,0,1,1,17,0,722,1504,sys-apps findutils install a single version of...,sys-apps findutils install a single version of...,0.702467,1,False,0.0,NaN,False,False,875.324375,748,1.409999e+09,1.410237e+09,238163.0,ABANDONED,0.000000,1002279,308,9,True,1,0
2,2,40,36,1,1,22,35,668,50796,make vp8lsetbitpos set br- eos_ flag,make vp8lsetbitpos set br- eos_ flag readsymbo...,0.588432,1,False,0.0,NaN,False,True,875.904201,1371,1.410000e+09,1.410000e+09,155.0,MERGED,45.666667,1000898,431,16,True,2,0
3,3,4,0,2,2,4,0,126,62508,upstream ieee80211 add definition for 802.11ac...,upstream ieee80211 add definition for 802.11ac...,0.000000,1,False,0.0,NaN,False,False,876.148218,7175,1.410002e+09,1.410311e+09,308512.0,MERGED,3.000000,1002363,1166,69,True,3,0
4,4,89,0,3,3,77,0,2066,2066,ddccontrol import newer version,ddccontrol import newer version this is the la...,0.395174,3,False,0.0,NaN,False,False,875.362002,749,1.410003e+09,1.410285e+09,282442.0,MERGED,0.666667,1000768,2125,13,True,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,315,1,2,2,266,1,7484,8537,charge_manager add unit tests,charge_manager add unit tests add unit tests f...,0.354284,1,False,0.0,NaN,False,False,946.725104,4778,1.416098e+09,1.416113e+09,15191.0,MERGED,41.166667,1103909,296,43,True,9,0
9996,9996,0,20,2,2,0,13,0,61396,update_engine remove unused methods from updat...,update_engine remove unused methods from updat...,0.934068,2,False,0.0,NaN,False,False,260.838414,1344,1.416098e+09,1.416299e+09,201517.0,MERGED,21.500000,1002279,377,27,True,12,0
9997,9997,82,9,4,4,73,9,2083,10690,autotest allow non-test provision actions lik...,autotest allow non-test provision actions lik...,0.751032,2,False,0.0,NaN,False,False,946.434387,8648,1.416112e+09,1.534285e+09,118172970.0,MERGED,7.000000,1001834,138,9,True,11,0
9998,9998,16,0,5,5,16,0,365,16553,chromium arm dts veyron add ts3a227e headset c...,chromium arm dts veyron add ts3a227e headset c...,0.000000,5,False,0.0,NaN,False,False,946.975521,10088,1.416122e+09,1.416448e+09,326707.0,MERGED,31.000000,1000994,1502,110,True,3,0


In [ ]:
status_list = list(only_determined.status.unique())
status_to_index = {}
index_to_status = {}

def make_status_categories():
  for status,i in enumerate(status_list):
    status_to_index[i] = status
    index_to_status[status] = i
  return status_to_index, index_to_status
status_to_index, index_to_status = make_status_categories()


def convert_status_to_code(row):
  row['status_code'] = status_to_index[row['status']]
  return row
only_determined = only_determined.apply(convert_status_to_code,axis=1).drop(['status'], axis=1)

In [ ]:
owner_list = list(only_determined.owner.unique())
owner_to_index = {}
index_to_owner = {}

def make_owner_categories():
  for owner,i in enumerate(owner_list):
    owner_to_index[i] = owner
    index_to_owner[owner] = i
  return owner_to_index, index_to_owner
owner_to_index, index_to_owner = make_owner_categories()


def convert_owner_to_code(row):
  row['owner_code'] = owner_to_index[row['owner']]
  return row
only_determined = only_determined.apply(convert_owner_to_code,axis=1).drop(['owner'], axis=1)

In [ ]:
only_determined['mergeable'] = only_determined['mergeable'].fillna(-1)

In [ ]:
only_determined

,index,additions,deletions,number_of_patches_so_far,initial_files_changed,max_files_addition,max_files_deletion,max_files_delta,max_files_size,commit_sbj,commit_msg,entropy,files_touched,has_topic,topic_prevalency,mergeable,has_assignee,owner_is_submitter,project_age,project_prev_commits,created,updated,eval_time,average_NUR,dev_prev_exp,dev_merged_CR,decision_made,project_code,topic_code,status_code,owner_code
0,0,2,5,2,2,2,5,0,2843,tegra132 fix compilation error,tegra132 fix compilation error two commits wer...,0.000000,1,False,0.0,-1.0,False,False,515.242245,3574,1.409997e+09,1.410232e+09,235082.0,3.000000,640,0,True,0,0,0,0
1,1,21,0,1,1,17,0,722,1504,sys-apps findutils install a single version of...,sys-apps findutils install a single version of...,0.702467,1,False,0.0,-1.0,False,False,875.324375,748,1.409999e+09,1.410237e+09,238163.0,0.000000,308,9,True,1,0,1,1
2,2,40,36,1,1,22,35,668,50796,make vp8lsetbitpos set br- eos_ flag,make vp8lsetbitpos set br- eos_ flag readsymbo...,0.588432,1,False,0.0,-1.0,False,True,875.904201,1371,1.410000e+09,1.410000e+09,155.0,45.666667,431,16,True,2,0,0,2
3,3,4,0,2,2,4,0,126,62508,upstream ieee80211 add definition for 802.11ac...,upstream ieee80211 add definition for 802.11ac...,0.000000,1,False,0.0,-1.0,False,False,876.148218,7175,1.410002e+09,1.410311e+09,308512.0,3.000000,1166,69,True,3,0,0,3
4,4,89,0,3,3,77,0,2066,2066,ddccontrol import newer version,ddccontrol import newer version this is the la...,0.395174,3,False,0.0,-1.0,False,False,875.362002,749,1.410003e+09,1.410285e+09,282442.0,0.666667,2125,13,True,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,315,1,2,2,266,1,7484,8537,charge_manager add unit tests,charge_manager add unit tests add unit tests f...,0.354284,1,False,0.0,-1.0,False,False,946.725104,4778,1.416098e+09,1.416113e+09,15191.0,41.166667,296,43,True,9,0,0,8
9996,9996,0,20,2,2,0,13,0,61396,update_engine remove unused methods from updat...,update_engine remove unused methods from updat...,0.934068,2,False,0.0,-1.0,False,False,260.838414,1344,1.416098e+09,1.416299e+09,201517.0,21.500000,377,27,True,12,0,0,1
9997,9997,82,9,4,4,73,9,2083,10690,autotest allow non-test provision actions lik...,autotest allow non-test provision actions lik...,0.751032,2,False,0.0,-1.0,False,False,946.434387,8648,1.416112e+09,1.534285e+09,118172970.0,7.000000,138,9,True,11,0,0,12
9998,9998,16,0,5,5,16,0,365,16553,chromium arm dts veyron add ts3a227e headset c...,chromium arm dts veyron add ts3a227e headset c...,0.000000,5,False,0.0,-1.0,False,False,946.975521,10088,1.416122e+09,1.416448e+09,326707.0,31.000000,1502,110,True,3,0,0,22


In [ ]:
def separate_data(data):
  text_columns = ['commit_sbj','commit_msg']
  clmns = data.columns
  for textual_clmn in text_columns:
    clmns = clmns[clmns!=textual_clmn]

  target_info = ['eval_time']
  for target_clmn in target_info:
    clmns = clmns[clmns!=target_clmn]

  future_clmns = ['number_of_patches_so_far','status','decision_made']
  for future_clmn in future_clmns:
    clmns = clmns[clmns!=future_clmn]

  numerical_inputs = data[clmns].astype('float')
  textual_inputs = data[text_columns]
  target = data[target_info]
  return textual_inputs, numerical_inputs, target

In [ ]:
def convert_target_to_hour(row):
  row['eval_time'] = row['eval_time']/3600
  return row
only_determined = only_determined.apply(convert_target_to_hour,axis=1).drop(['created','updated'], axis=1)

In [ ]:
def evaluateImageModel(model, data, criterion, device='cuda',epoch=1):
    model.eval()
    total_loss = 0.0
    total_loss = 0.
    start_time = time.time()
    textual_inputs, numerical_inputs, targets = separate_data(data)
    textual_inputs = (textual_inputs.values.tolist())
    # print(len(textual_inputs))
    # print(textual_inputs)
    numerical_inputs = numerical_inputs.values.tolist()
    targets = targets.values.tolist()
    all_losses = []
    with torch.no_grad():
      for batch, i in tqdm(enumerate(range(0, only_determined.shape[0] - 1, 1)),total=only_determined.shape[0], disable=True):
        numerical_inpt=torch.tensor(numerical_inputs[i],dtype=torch.float).cuda()
        output = model(textual_inputs[i],numerical_inpt)
        target= torch.tensor(targets[i],dtype=torch.float).cuda()
        loss = criterion(output, target)
        total_loss += loss.item()
        elapsed = time.time() - start_time
        print(textual_inputs[i],numerical_inpt)
        print(output,target,loss.item())
        if i % save_round == 0:
          # print('\n| epoch ',epoch, '| {:5d}/{:5d} \t| {:5.2f}\t | loss {:5.2f}'
          #       .format(i+1, len(data) // bptt, elapsed * 1000 ,total_loss / save_round))
          all_losses.append(total_loss / save_round)
          total_loss = 0
          start_time = time.time()

    return all_losses


In [ ]:
evaluateImageModel(model,only_determined.iloc[1:1000],criterion)

['sys-apps findutils install a single version of find.', "sys-apps findutils install a single version of find. findutils implements two versions of the same find tool the fts find find ftsfind.c and the old find find find.c that are 100% compatible. one of the versions gets installed as find while the other version gets installed as oldfind or ftsfind , based on the --without-fts flag during configure. this patch removes the other version from the makefile.in file so it doesn't get compiled nor installed. bug chromium 411406 test emerge-link findutils test `equery-link f findutils` doesn't show oldfind. change-id ie4b2434bf31d5dadbbe0593e0492c2e8dc900fa2 "] tensor([ 1.0000e+00,  2.1000e+01,  0.0000e+00,  1.0000e+00,  1.7000e+01,
         0.0000e+00,  7.2200e+02,  1.5040e+03,  7.0247e-01,  1.0000e+00,
         0.0000e+00,  0.0000e+00, -1.0000e+00,  0.0000e+00,  0.0000e+00,
         8.7532e+02,  7.4800e+02,  0.0000e+00,  3.0800e+02,  9.0000e+00,
         1.0000e+00,  0.0000e+00,  1.0000e

KeyboardInterrupt: ignored